# Setting some useful variables

In [1]:
import os
import re
APP_NAME="myapp"
ROOT_PATH=os.path.abspath(".")
print "ROOT_PATH: %s"%ROOT_PATH

input_filename=os.path.join(ROOT_PATH,"rent_input.xls")
print input_filename
import datetime

ROOT_PATH: C:\Users\Wasit\Documents\GitHub\cs459_final\myproject
C:\Users\Wasit\Documents\GitHub\cs459_final\myproject\rent_input.xls


# Loading  Car detail from the 1st sheet from the input excel

In [2]:
import pandas as pd
cvt={
        0:int,
        1:unicode,
        2:unicode,
        3:float,
        4:unicode,
        5:int,
    }
df_car=pd.read_excel(io=input_filename,sheetname=0,converters=cvt)

In [3]:
print df_car

   ID    CarMaker CarModel  CarPrice   CarColor  CarYear
0   1  Mitsubishi     L200    9995.0        red     2001
1   2        Mini   Cooper   12500.0        red     2005
2   3         TVR   Tuscan   18000.0       blue     2003
3   4         BMW       Z3   13995.0     silver     2002
4   5      Toyota   Celica    4665.0  dark blue     1997
5   6        Audi       TT   21995.0     silver     2005
6   7    Mercedes     E320   15495.0      green     2004


# Example for iterate over all row in the sheet

In [4]:
for k,i in df_car.iterrows():
    print "k: %s, i:%s\n%s"%(k, i, "-"*30)

k: 0, i:ID                   1
CarMaker    Mitsubishi
CarModel          L200
CarPrice          9995
CarColor           red
CarYear           2001
Name: 0, dtype: object
------------------------------
k: 1, i:ID               2
CarMaker      Mini
CarModel    Cooper
CarPrice     12500
CarColor       red
CarYear       2005
Name: 1, dtype: object
------------------------------
k: 2, i:ID               3
CarMaker       TVR
CarModel    Tuscan
CarPrice     18000
CarColor      blue
CarYear       2003
Name: 2, dtype: object
------------------------------
k: 3, i:ID               4
CarMaker       BMW
CarModel        Z3
CarPrice     13995
CarColor    silver
CarYear       2002
Name: 3, dtype: object
------------------------------
k: 4, i:ID                  5
CarMaker       Toyota
CarModel       Celica
CarPrice         4665
CarColor    dark blue
CarYear          1997
Name: 4, dtype: object
------------------------------
k: 5, i:ID               6
CarMaker      Audi
CarModel        TT
CarPrice     

# Each row contains 6 columns as following

In [5]:
i

ID                 7
CarMaker    Mercedes
CarModel        E320
CarPrice       15495
CarColor       green
CarYear         2004
Name: 6, dtype: object

# Uploading the row i into database

In [6]:
import pytz
year=datetime.datetime(year=i['CarYear'], month=1, day=1, tzinfo=pytz.UTC)
kargs={
    'maker':i['CarMaker'], 
    'price':i['CarPrice'], 
    'model':i['CarModel'], 
    'year': year
}
car, created = Car.objects.update_or_create(
                    id=i['ID'],
                    defaults=kargs
                )

# Now uploading every rows

In [7]:
for k,i in df_car.iterrows():
    year=datetime.datetime(year=i['CarYear'], month=1,day=1, tzinfo=pytz.UTC)
    kargs={
        'maker':i['CarMaker'], 
        'price':i['CarPrice'], 
        'model':i['CarModel'], 
        'year': year
    }
    car, created = Car.objects.update_or_create(
                        id=i['ID'],
                        defaults=kargs
                    )

# Then loading customer from the 2nd sheet

In [8]:
import pandas as pd
cvt={
        0:unicode,
        1:unicode,
        2:unicode,
        3:unicode,
        4:unicode,
        5:unicode,
        6:unicode
    }
df_customer=pd.read_excel(io=input_filename,sheetname=1,converters=cvt)
keys=df_customer.keys()
print keys

Index([u'ID', u'ClientFirstName', u'ClientLastName', u'ClientAddress',
       u'Postcode', u'Tel', u'Email'],
      dtype='object')


# And uploading the customer

In [9]:
for k,i in df_customer.iterrows():
    kargs={
        'first_name':i[keys[1]], 
        'last_name':i[keys[2]], 
        'Address':i[keys[3]], 
        'postcode':i[keys[4]], 
        'telephone':i[keys[5]], 
        'email':i[keys[6]]
    }
    customer, created = Customer.objects.update_or_create(
                        id=i['ID'],
                        defaults=kargs
                    )

# Finally Loading the 3rd sheet

In [10]:
import pandas as pd
df_rent=pd.read_excel(io=input_filename,sheetname=2)
keys=df_rent.keys()
print keys

Index([u'ID', u'RentDate', u'ServiceCost', u'ReturnDate', u'ClientID',
       u'CarID'],
      dtype='object')


In [11]:
df_rent

,ID,RentDate,ServiceCost,ReturnDate,ClientID,CarID
0,1,2014-03-12,549.75,2014-03-17,1,1
1,2,2014-03-12,1050.00,2014-03-20,2,2
2,3,2014-03-13,1310.00,2014-03-20,3,3
3,4,2014-03-17,425.00,2014-03-20,1,2
4,5,2014-03-20,189.95,2014-03-21,4,4
5,6,2014-03-20,50.00,2014-03-20,2,5
6,7,2014-03-20,269.95,2014-03-21,2,6
7,8,2014-03-21,514.85,2014-03-24,5,7
8,9,2014-03-24,549.75,2014-03-29,6,1
9,10,2014-03-29,50.00,2014-03-29,1,2


# And uploading  Rent records to the database

In [12]:
for k,i in df_rent.iterrows():
    utc=pytz.timezone('UTC')
    kargs={
        'rent_date': utc.localize(i['RentDate']),
        'return_date':utc.localize(i['ReturnDate']), 
        'cost':i['ServiceCost'], 
        'car': Car.objects.get(id=i['CarID']), 
        'customer': Customer.objects.get(id=i['ClientID']), 
    }
    customer, created = Rent.objects.update_or_create(
                        id=i['ID'],
                        defaults=kargs
                    )

# Code explain

To convert from naive-datetime to time-zone-aware-datetime
```python
import pytz
utc=pytz.timezone('UTC')
utc.localize( your_datetime )
```